In [ ]:
import nltk
import xml.etree.ElementTree as ET
from difflib import SequenceMatcher
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/suyamoon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def similarity(a, b):
    """Calculate text similarity percentage using SequenceMatcher"""
    return SequenceMatcher(None, a, b).ratio() * 100

In [ ]:
def normalize_text(text):
    """Normalize text for better matching"""
    if text is None:
        return ""
    return ' '.join(text.lower().split())

In [ ]:
def process_files(xml_path, txt_path, output_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    arguments = []
    for elem in root:
        if elem.tag == 'prem':
            arguments.append({
                'type': 'P',
                'id': elem.attrib['ID'],
                'text': elem.text.strip() if elem.text else ""
            })
        elif elem.tag == 'conc':
            arguments.append({
                'type': 'C',
                'id': elem.attrib['ID'],
                'text': elem.text.strip() if elem.text else ""
            })
    
    for arg in arguments:
        arg['normalized_text'] = normalize_text(arg['text'])
    
    with open(txt_path, 'r', encoding='latin-1') as f:
        full_text = f.read()
    
    sentences = sent_tokenize(full_text)
    
    processed_sentences = []
    matched_args = set()
    
    for i, sentence in enumerate(sentences):
        processed_sentences.append({'text': sentence, 'tag': 'NA', 'matched': False})
    
    for window_size in range(5, 0, -1):  
        for i in range(len(sentences) - window_size + 1):
            window_text = ' '.join(sentences[i:i+window_size])
            normalized_window = normalize_text(window_text)
            
            best_match = None
            best_similarity = 0
            
            for arg in arguments:
                if arg['id'] in matched_args:
                    continue 
                
                sim = similarity(normalized_window, arg['normalized_text'])
                if sim > best_similarity:
                    best_similarity = sim
                    best_match = arg
            
            if best_similarity >= 90:
                matched_args.add(best_match['id'])
                for j in range(i, i+window_size):
                    processed_sentences[j]['tag'] = f"{best_match['type']}_{best_match['id']}"
                    processed_sentences[j]['matched'] = True
                print(f"Matched {best_match['type']}_{best_match['id']} with {best_similarity:.2f}% similarity")
            elif best_match and best_similarity >= 70:
                print(f"Low match ({best_similarity:.2f}%): {window_text[:100]}... with {best_match['type']}_{best_match['id']}")
    
    for arg in arguments:
        if arg['id'] in matched_args:
            continue 
        
        best_match_idx = -1
        best_similarity = 0
        
        for i, sent_info in enumerate(processed_sentences):
            if sent_info['matched']:
                continue
            
            sim = similarity(normalize_text(sent_info['text']), arg['normalized_text'])
            if sim > best_similarity:
                best_similarity = sim
                best_match_idx = i
        
        if best_similarity >= 90:
            processed_sentences[best_match_idx]['tag'] = f"{arg['type']}_{arg['id']}"
            processed_sentences[best_match_idx]['matched'] = True
            matched_args.add(arg['id'])
            print(f"Matched {arg['type']}_{arg['id']} with {best_similarity:.2f}% similarity (single sentence)")
        elif best_match_idx >= 0 and best_similarity >= 70:
            print(f"Low match ({best_similarity:.2f}%): {processed_sentences[best_match_idx]['text'][:100]}... with {arg['type']}_{arg['id']}")
    
    output_lines = []
    for sent_info in processed_sentences:
        output_lines.append(f"{sent_info['text']} || {sent_info['tag']}")
    
    with open(output_path, 'w', encoding='latin-1') as f:
        f.write('\n'.join(output_lines))
    
    print(f"Processing complete. Output written to {output_path}")
    print(f"Matched {len(matched_args)} out of {len(arguments)} arguments")
    
    unmatched = [f"{arg['type']}_{arg['id']}" for arg in arguments if arg['id'] not in matched_args]
    if unmatched:
        print(f"Unmatched arguments: {', '.join(unmatched)}")


In [ ]:
process_files('A2017_European Commission v Italian Republic_DT.xml', 'A2017_European Commission v Italian Republic_DT.txt', 'A2017_European Commission v Italian Republic_DT___annotated_judgment.txt')


Low match (81.94%): In the originating application that issue was raised by that Member State only in relation to the al... with P_A1
Low match (73.09%): 15      While certain pleas may, and indeed must, be raised by the courts of their own motion, such ... with P_A2
Low match (74.13%): 36      The Court must therefore ascertain whether, as the Commission claims, the General Court’s re... with P_B10
Low match (71.53%): 42      However, by legge n. 10, Conversione in legge, con modificazioni, del decreto-legge 29 dicem... with P_B16
Low match (73.88%): In the light of the finding in paragraph 43 above, that scenario does not concern the alteration at ... with P_B33
Low match (72.50%): It did not, however, examine the effects of that alteration on the original aid scheme. 53      The ... with P_B35
Low match (71.38%): 55      Therefore, the General Court, having held, in paragraphs 81 and 82 of the judgment under app... with P_B38
Matched P_A1 with 91.39% similarity
Low match (82.60%): 1

In [14]:
process_files('A2013_European Commission v Ireland and Others.xml', 'A2013_European Commission v Ireland and Others.txt', 'A2013_European Commission v Ireland and Others___annotated_judgment.txt')


Low match (73.61%): Findings of the Court

27      It follows from the rules governing the procedure before the Courts o... with P_A2
Low match (72.92%): 34      Yet, as submitted by the Commission and as stated by the Advocate General in points 57 to 63... with P_A10
Low match (73.49%): Although the General Court may have linked the question of the attributability of the exemptions at ... with P_A12
Low match (77.28%): It is therefore necessary to examine whether that judgment remains well founded on the basis of the ... with P_A17
Low match (70.22%): 43      Last, after finding, in paragraphs 100 to 103 of the judgment under appeal, that the Commiss... with P_A24
Low match (79.58%): The purpose and the scope of the procedure laid down in that article differ from those of the rules ... with P_A31
Low match (70.14%): Articles 87 EC and 88 EC thus reserve a central role for the Commission in determining whether aid i... with P_A34
Low match (71.28%): 54      It follows that the grounds 

In [16]:
process_files('A2018_Commission v Spain.xml', 'A2018_Commission v Spain.txt', 'annotated/A2018_Commission v Spain___annotated_judgment.txt')


Low match (72.82%): That complaint is therefore admissible. 33      With regard to the Commission’s argument that the EI... with P_A6
Low match (78.84%): First, there must be an intervention by the State or through State resources. Second, the interventi... with P_A10quinquies
Low match (73.68%): 32      By that complaint, the Commission does not dispute the facts on which the General Court reli... with P_A3
Low match (75.23%): In doing so, the Commission seeks a review of the General Court’s legal classification relating to t... with P_A6
Low match (86.32%): That complaint is therefore admissible. 33      With regard to the Commission’s argument that the EI... with P_A6
Low match (75.51%): 33      With regard to the Commission’s argument that the EIG and its members are part of an economi... with P_A6
Low match (73.68%): 35      According to the settled case-law of the Court of Justice, classification of a national meas... with P_A10
Low match (85.27%): Second, the intervention must b

In [18]:
process_files('R2013_Telefónica SA v European Commission.xml', 'R2013_Telefónica SA v European Commission.txt', 'annotated/R2013_Telefónica SA v European Commission___annotated_judgment.txt')


Low match (70.02%): 58

As is clear from paragraphs 34 to 36 of the present judgment, the contested decision entails
imp... with P_C5
Low match (78.81%): 59

Consequently, even though Telefónica cannot, because of the conditions governing admissibility l... with P_C5
Low match (78.05%): 38

The General Court was therefore correct in holding, in paragraph 45 of the order under appeal, t... with P_A19
Low match (70.44%): 47

As the General Court pointed out in paragraph 28 of the order under appeal, it is also clear fro... with P_B4
Low match (71.73%): 58

As is clear from paragraphs 34 to 36 of the present judgment, the contested decision entails
imp... with P_C5
Low match (79.04%): 59

Consequently, even though Telefónica cannot, because of the conditions governing admissibility l... with P_C5
Matched P_A6 with 90.27% similarity
Low match (89.60%): Where that implementation is a matter for the Member

ECLI:EU:C:2013:852

5

JUDGMENT OF 19. 12. 20... with P_A7
Low match (78.11%): 34

F

In [19]:
process_files('R2016_Hellenic Republic v European Commission.xml', 'R2016_Hellenic Republic v European Commission.txt', 'annotated/R2016_Hellenic Republic v European Commission___annotated_judgment.txt')


Low match (75.24%): 49   Lastly, the Hellenic Republic submits, in the alternative, that the General Court failed to exa... with P_B1
Low match (76.85%): 58   It has already been held in paragraph 41 of this judgment that the statement of reasons in the
... with P_B11
Low match (76.43%): Accordingly, the General Court was




     ECLI:EU:C:2016:145                                      ... with P_A37
Low match (75.43%): 49   Lastly, the Hellenic Republic submits, in the alternative, that the General Court failed to exa... with P_B1
Low match (84.18%): 50   The Commission disputes the admissibility as well as the merits of the present ground of appeal... with P_B1
Low match (76.96%): 58   It has already been held in paragraph 41 of this judgment that the statement of reasons in the
... with P_B11
Matched P_B11 with 90.75% similarity
Low match (76.57%): 12                                                                                  ECLI:EU:C:2016:1... with P_C5
Low match (70.25%): Fu

In [24]:
process_files('R2016_Netherlands Maritime Technology Association formerly Scheepsbouw Nederland v European Commission.xml', 'R2016_Netherlands Maritime Technology Association formerly Scheepsbouw Nederland v European Commission.txt', 'annotated/R2016_Netherlands Maritime Technology Association formerly Scheepsbouw Nederland v European Commission___annotated_judgment.txt')


Low match (76.12%): 38 The Commission argues that this ground of appeal is unfounded. 39 The Kingdom of Spain contends t... with P_A1
Low match (73.83%): Further, in paragraphs 93 to 101

of that judgment, the General Court set out the reasons why it hel... with P_A21
Low match (76.61%): Since the General Court is not required to reply exhaustively to each of the arguments put forward b... with P_A23
Low match (78.06%): In particular, the General Court set out, in paragraph 91 of that
judgment, the reasons that led the... with P_B7
Low match (72.67%): 60 Further, the obligation to state reasons does not require the General Court to provide an account... with P_B7
Low match (74.69%): The General Court, in paragraph 91 of that judgment, rejected the appellant's arguments as to the in... with P_C15
Low match (73.24%): 77 Accordingly, the second part of the third ground of appeal is unfounded. 78 As regards the third ... with P_C18
Low match (71.14%): 84 Accordingly, the third part of the 

In [25]:
process_files('R2016_European Commission v Hansestadt Lübeck.xml', 'R2016_European Commission v Hansestadt Lübeck.txt', 'annotated/R2016_European Commission v Hansestadt Lübeck___annotated_judgment.txt')


Low match (71.71%): 29  In that regard, it should be recalled that the Court of Justice has also held, in the judgment o... with P_B8
Low match (70.86%): The concept of selectivity, as the Advocate General states in point 75 of his Opinion, is thus linke... with P_C18
Low match (74.16%): In this regard, the General Court's assessment conflicts with that resulting from the judgment of 10... with P_A1
Low match (76.73%): 14 The City of Lübeck and the Federal Republic of Germany contend that this ground of appeal should ... with P_A1
Low match (70.29%): Thus, according to the General Court, the power to adopt the 2006 schedule lay with FL and not the S... with P_A8
Low match (73.50%): 19  It is apparent that the General Court thus, in the light of the applicable national law, took th... with P_A8
Low match (79.65%): 21 In the present instance, the Commission has not pleaded and, a fortiori, has not demonstrated suc... with P_A10
Low match (71.31%): It stated that such a decision necessari

In [26]:
process_files('R2016_Orange v European Commission.xml', 'R2016_Orange v European Commission.txt', 'annotated/R2016_Orange v European Commission___annotated_judgment.txt')


Low match (70.92%): 44  Indeed, it is clear that, to date, the only situation recognised by the Court's case-law in whic... with P_A27
Low match (73.38%): 20  Therefore, according to Orange, the General Court erred in law by endorsing the Commission's cho... with P_A1
Low match (70.04%): 26 That assessment is not vitiated by the errors of law alleged by Orange in the second argument of ... with P_A7
Low match (76.04%): 39 It follows that the third argument of the first part of the first ground of appeal must be reject... with P_A22
Low match (71.61%): Indeed, according to Orange, compensation for a structural disadvantage may preclude the categorisat... with P_A25
Low match (74.73%): 41 The General Court rejected that argument in paragraphs 42 and 43 of the judgment under appeal, st... with P_A25
Low match (74.99%): 44  Indeed, it is clear that, to date, the only situation recognised by the Court's case-law in whic... with P_A27
Low match (73.11%): 45 The General Court was therefore en

In [27]:
process_files('R2017_European Commission v Frucona Košice a.xml', 'R2017_European Commission v Frucona Košice a.txt', 'annotated/R2017_European Commission v Frucona Košice a___annotated_judgment.txt')


Low match (79.44%): However, the decision at issue makes no mention of such a request or such an assessment. 37 Lastly, ... with P_B5
Low match (77.61%): 77 As the Advocate General noted in paragraphs 125 and 131 of his Opinion, the General Court merely ... with P_D20
Low match (74.60%): 82 Furthermore, it follows from paragraphs 69 to 84 of the present judgment that, contrary to the Co... with P_D26
Low match (79.33%): 18  Frucona Koice contests the merits of the Commission's arguments. Findings of the Court
19 Accor... with P_A1
Low match (73.74%): 21 However, the conditions which a measure must meet in order to be treated as 'aid' for the purpose... with P_A4
Low match (70.66%): 22 When a public creditor grants payment facilities in respect of a debt payable to it by an underta... with P_A5
Low match (76.13%): 24 As a result, where it appears that the private creditor test might be applicable, it is for the C... with P_A8
Low match (74.06%): 26 Accordingly, as the Advocate General 

In [28]:
process_files('R2021_FVE Holýšov I and Others v Commission.xml', 'R2021_FVE Holýšov I and Others v Commission.txt', 'annotated/R2021_FVE Holýšov I and Others v Commission___annotated_judgment.txt')


Low match (70.42%): The
appellants have not shown that that evidence was sufficient to justify the legitimate expectatio... with P_C11
Low match (73.11%): 23 That assessment by the General Court of the facts and evidence before it cannot be challenged bef... with P_A3
Low match (70.71%): The appellants stress that, even without the 2004 letter, the Commission was bound by its 2006 decis... with P_B1
Low match (84.90%): 28 The Commission contends that the second ground of appeal should be rejected. Findings of the Cour... with P_B1
Low match (71.48%): Findings of the Court
29 The plea in the application at first instance alleging that the Commission ... with P_B1
Low match (70.43%): 30 It is common ground that that plea was only put forward for the first time in the reply. Furtherm... with P_B3
Low match (81.89%): The
appellants have not shown that that evidence was sufficient to justify the legitimate expectatio... with P_C11
Low match (77.12%): 39 In particular, the appellants do not 

In [29]:
process_files('R2021_Prosegur Compañía de Seguridad SA, established in Madrid (Spain) v Commission.xml', 'R2021_Prosegur Compañía de Seguridad SA, established in Madrid (Spain) v Commission.txt', 'annotated/R2021_Prosegur Compañía de Seguridad SA, established in Madrid (Spain) v Commission___annotated_judgment.txt')


Low match (70.35%): The General Court did not err in law in concluding therefrom, in paragraph 205 of the judgment under... with P_H10
Low match (71.53%): In those circumstances, the appellant is entitled to challenge, on appeal, the grounds of the judgme... with P_A7
Low match (73.17%): 41 Furthermore, as the Advocate General observed in point 35 of his Opinion in Joined Cases World Du... with P_A7
Low match (76.84%): 42 In the light of those considerations, the first part the single ground of appeal is admissible. (... with P_B2
Low match (73.17%): In that regard, the Court has held on numerous occasions that the objective pursued by measures of S... with P_B11
Low match (74.73%): 50 It is in the light of those considerations that the Court must assess the merits of the appellant... with P_B15
Low match (79.37%): The appellant refers in particular to paragraphs 82 and 130 of the judgment under appeal. 53 In that... with P_B17
Low match (70.99%): However, in recital 89 of that decisio

In [31]:
process_files('R2006_European Commission v Italian Republic.xml', 'R2006_European Commission v Italian Republic.txt', 'annotated/R2006_European Commission v Italian Republic___annotated_judgment.txt')


Processing complete. Output written to annotated/R2006_European Commission v Italian Republic___annotated_judgment.txt
Matched 0 out of 37 arguments
Unmatched arguments: P_A1, P_A2, P_A3, P_A4, P_A5, P_A6, P_A7, P_A8, P_A9, P_A10, C_A11, P_B1, P_B2, P_B3, P_B4, P_B5, P_B6, P_B7, P_B8, P_B9, P_B10, P_B11, P_B12, P_B13, P_B14, P_B15, P_B16, P_B17, P_B18, P_B19, P_B20, P_B21, P_B22, P_B23, P_B24, P_B25, C_B26


In [32]:
process_files('pdf/A2009_Commission of the European Communities v Koninklijke FrieslandCampina NV_M.xml', 'pdf/A2009_Commission of the European Communities v Koninklijke FrieslandCampina NV_M.txt', 'annotated/A2009_Commission of the European Communities v Koninklijke FrieslandCampina NV_M___annotated_judgment.txt')


Low match (71.35%): In that regard, it should be recalled that KFC was
part of a group of, at most, 14 applicants for fi... with P_A10
Low match (71.07%): 64  
In the present case, it is not in dispute that, if the contested decision had extended the
tran... with P_B2
Low match (89.12%): Findings of the Court 
47  
Under the fourth paragraph of Article 230 EC, a natural or legal person ... with P_A1
Low match (81.45%): In that regard, it should be recalled that KFC was
part of a group of, at most, 14 applicants for fi... with P_A10
Low match (88.02%): Thus, as the Court of First Instance stated correctly in paragraphs 98 and 100 of the
judgment under... with P_A11
Low match (70.76%): The 
Commissionâs knowledge of the situation of the undertakings which had requested first 
GFA au... with P_A17
Low match (71.33%): 64  
In the present case, it is not in dispute that, if the contested decision had extended the
tran... with P_B2
Matched P_B3 with 93.72% similarity
Low match (71.01%): In

In [33]:
process_files('pdf/A2011_European Commission (C-106_09 P) and Kingdom of Spain (C-107_09 P) v Government of Gibraltar and United Kingdom of Great Britain and Northern Ireland.xml', 'pdf/A2011_European Commission (C-106_09 P) and Kingdom of Spain (C-107_09 P) v Government of Gibraltar and United Kingdom of Great Britain and Northern Ireland.txt', 'annotated/A2011_European Commission (C-106_09 P) and Kingdom of Spain (C-107_09 P) v Government of Gibraltar and United Kingdom of Great Britain and Northern Ireland___annotated_judgment.txt')


Low match (77.09%): That approach is therefore at variance with the case-law cited in para
graphÂ 87 above. 89
Secondly,... with P_B7
Low match (74.63%): 97
It is true that, in the absence of European Union rules governing the matter, it falls 
within th... with P_B17
Low match (71.33%): 107
Thus, the fact that offshore companies, which constitute a group of companies with 
regard to th... with P_B27
Matched P_A7 with 90.82% similarity
Low match (74.36%): 11. 2011 â JOINED CASES C-106/09Â P AND C-107/09Â P
C-143/99 Adria-Wien Pipeline and Wietersdorfer... with P_A8
Low match (71.71%): (a)Â The requirement to make a profit and the capping of tax
77
The General Court concluded with reg... with P_A13
Matched P_A15 with 91.11% similarity
Low match (82.98%): Iâ -â 11208
JUDGMENT OF 15. 11. 2011 â JOINED CASES C-106/09Â P AND C-107/09Â P
80
The require... with P_A16
Low match (77.58%): That approach is therefore at variance with the case-law cited in para
graphÂ 87 above. 89
Secondly

In [34]:
process_files('pdf/A2012__BNP Paribas and Banca Nazionale del Lavoro SpA (BNL) v European Commission.xml', 'pdf/A2012__BNP Paribas and Banca Nazionale del Lavoro SpA (BNL) v European Commission.txt', 'annotated/A2012__BNP Paribas and Banca Nazionale del Lavoro SpA (BNL) v European Commission___annotated_judgment.txt')


Low match (73.63%): It follows that the General Court was 
required to respond to them. 67 In addition, in paragraph 182... with P_A20
Low match (72.94%): However, the 
General Court did not find that this was the case here. 104 Thus, by merely observing ... with P_C17
Low match (77.39%): The fact that, in the 
absence of the realignment scheme at issue, the undertakings concerned would ... with P_A10
Low match (73.85%): It follows that the General Court was 
required to respond to them. 67 In addition, in paragraph 182... with P_A20
Low match (78.89%): 67 In addition, in paragraph 182 of the judgment under appeal, the General Court held that there was... with P_A20
Low match (84.71%): However, the 
General Court did not find that this was the case here. 104 Thus, by merely observing ... with P_C17
Low match (77.30%): 104 Thus, by merely observing that the contested decision did not concern the transferring entities ... with P_C17
Matched P_A4 with 93.87% similarity
Low match (77.59%):

In [35]:
process_files('pdf/R2021_World Duty Free v. Commission.xml', 'pdf/R2021_World Duty Free v. Commission.txt', 'annotated/R2021_World Duty Free v. Commission___annotated_judgment.txt')


Low match (72.70%): 85
Consequently, as the Advocate General observed in point 65 of his Opinion, contrary to the 
appel... with P_B39
Low match (74.10%): ECLI:EU:C:2021:793                                                                                  ... with P_A3
Low match (86.37%): 79
It follows that, in paragraphs 70, 92, 123 and 140 of the judgment under appeal, the General Cour... with P_B32
Low match (72.07%): 84
Although, as the appellants submit, the Commission did indeed refer, in recitals 93 to 96 and 117... with P_B39
Low match (86.12%): 85
Consequently, as the Advocate General observed in point 65 of his Opinion, contrary to the 
appel... with P_B39
Low match (70.33%): Thus, the arguments advanced by the appellants in 
order to challenge the definition of the objectiv... with P_B48
Low match (72.84%): 113 Thus, with respect to the assessment in the context of an appeal of the General Courtâs findin... with P_C7
Low match (74.50%): Similarly, the statement that the tax